In [4]:
import ee
import numpy as np
import pandas as pd

In [5]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWinn3oTGENlYTJu9kOCIbGqyNLaA6FV7fm3OMSiwl40OFPY-MW-uPs



Successfully saved authorization token.


In [6]:
ee.Initialize()

In [7]:
# Make a band corresponding to coded input for MODIS LAI algorithm from MOD09 data
# Code is AABBCCDDEEFF as a 64bit integer
#
# AA is LC_Type3 Land Cover Type 3: Annual Leaf Area Index (LAI) classification
# BB is B01 quantized to 1% reflectance
# CC is B02 quantized to 1% reflectance
# DD is SensorZenith quantized to 1degree
# EE is Solar Zenith quantized to 1degree
# FF is relative azimuth quantized to 2 degree 
def getCodeMOD09(imageMOD09,imageMOD12):

    relAzimuth = imageMOD09.select('SensorAzimuth').subtract(imageMOD09.select('SolarAzimuth')).abs().max(imageMOD09.select('SensorAzimuth').subtract(imageMOD09.select('SolarAzimuth')).abs()).multiply(0.01)
    relAzimuth = relAzimuth.min(ee.Image.constant(360).subtract(relAzimuth)).multiply(0.5).round().multiply(1000000) 
    codeImage = relAzimuth.add(imageMOD09.select('SolarZenith').multiply(0.01).round().multiply(10000)) \
                          .add(imageMOD09.select('SensorZenith').multiply(0.01).round().multiply(100000000)) \
                          .add(imageMOD09.select('sur_refl_b01').min(ee.Image.constant(4950)).multiply(0.02).round().multiply(1)) \
                          .add(imageMOD09.select('sur_refl_b02').min(ee.Image.constant(4950)).multiply(0.02).round().multiply(100)) \
                          .add(imageMOD12.select('LC_Type3').round().multiply(10000000000)) \
                          .toInt64() \
                          .rename('codeMCD15')
                            
    return codeImage

In [8]:
#function to compute distance from images of pixel coordinates to a line defined by a list to two points
def getDist(image,pts) :
  
    # cast inputs
    image = ee.Image(image)
    pts = ee.List(pts)
    
    # identify coord layers
    x0 = image.select('x')
    y0 = image.select('y')
    
    # identify end points
    x1 = ee.Image.constant(ee.Number(ee.List(pts.get(0)).get(0)))
    x2 = ee.Image.constant(ee.Number(ee.List(pts.get(1)).get(0)))
    y1 = ee.Image.constant(ee.Number(ee.List(pts.get(0)).get(1)))
    y2 = ee.Image.constant(ee.Number(ee.List(pts.get(1)).get(1)))
    denom = ((x2.subtract(x1)).pow(ee.Image.constant(2)).add((y2.subtract(y1)).pow(ee.Image.constant(2)))).pow(ee.Image.constant(0.5))
    
    return ((((x2.subtract(x1)).multiply(y1.subtract(y0))).subtract((x1.subtract(x0)).multiply(y2.subtract(y1)))).abs()).divide(denom)
    



In [9]:
#  Estimate view zenith angle in a Landsat image based on lateral distance from satellite track
#  Assume flat earth and linear track within image
def getVZALC(imageLC) :

    # get point farthest from centroid p1
    points = ee.List(((imageLC.geometry().coordinates()).get(0)))
    dists0 = points.map( lambda point: ee.Geometry.Point(point).distance(imageLC.geometry().centroid()).multiply(-1))
    p1 = ee.List(points.sort(dists0).get(0))
    #print(p1)

    # get point farthest from p1:  p2
    dists1 = points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p1)).multiply(-1))
    p2 = ee.List(points.sort(dists1).get(0))
    #print(p2)

    # get point farthest from p1 and p2: p3
    dists2 =  points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p2)).multiply(-1))
    dists2 =  ee.List.sequence(0,dists1.length().subtract(1),1).map(lambda index: ee.Number(dists1.get(index)).add(ee.Number(dists2.get(index))) )
    p3 = ee.List(points.sort(dists2).get(0))
    #print(p3)

    # get point farthest from p3: p4
    dists3 =  points.map( lambda point: ee.Geometry.Point(point).distance(ee.Geometry.Point(p3)).multiply(-1))
    p4 = ee.List(points.sort(dists3).get(0))
    #print(p4)

    # separate easting and northing coords of points
    ptsE = [p1.get(0) , p2.get(0), p3.get(0), p4.get(0)]
    ptsN = [p1.get(1) , p2.get(1), p3.get(1), p4.get(1)]
    #print(ptsE)
    #print(ptsN)

    # make list of corner points
    pts = [p1,p2,p3,p4]
    #print(pts)

    # separate East and West side corner points
    ptWest = ee.Geometry.Point(ee.List(pts).sort(ptsE).get(0)).transform(imageLC.projection()).coordinates()
    ptNorth = ee.Geometry.Point(ee.List(pts).sort(ptsN).get(3)).transform(imageLC.projection()).coordinates()
    ptSouth = ee.Geometry.Point(ee.List(pts).sort(ptsN).get(0)).transform(imageLC.projection()).coordinates()
    ptEast = ee.Geometry.Point(ee.List(pts).sort(ptsE).get(3)).transform(imageLC.projection()).coordinates()

    ptsWest = ee.List([ptWest, ptNorth])
    ptsEast = ee.List([ptEast ,ptSouth])
    #print(ptsEast)
    #print(ptsWest )
    
    # compute distance between each pixel and line defined by East side corner points 
    imageLCCoords = ee.Image.pixelCoordinates(imageLC.projection()).clip(imageLC.geometry())
    distEast = getDist(imageLCCoords ,ptsEast)
    distWest = getDist(imageLCCoords ,ptsWest)

    # compute VZA assuming +/-7.5degree scan angle 
    return (distEast.divide(distEast.add(distWest)).subtract(ee.Image.constant(0.5)).multiply(ee.Image.constant(7.5)))

In [10]:
# coded LUT for output MODIS MCD15
def getLUTout(imageMCD15MOD09,satFlag):
    
     return imageMCD15MOD09.select('Lai').min(99).multiply(1).toInt64() \
                        .add(imageMCD15MOD09.select('LaiStdDev').min(99).multiply(1e2).toInt64()) \
                        .add(imageMCD15MOD09.select('Fpar').min(99).multiply(1e4).toInt64()) \
                        .add(imageMCD15MOD09.select('FparStdDev').min(99).multiply(1e6).toInt64()) \
                        .updateMask(imageMCD15MOD09.select('FparLai_QC').bitwiseAnd(1<<5).eq(satFlag)) \
                        .updateMask(imageMCD15MOD09.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                        .updateMask(imageMCD15MOD09.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) \
                        .updateMask(imageMCD15MOD09.select('FparExtra_QC').bitwiseAnd(1<<1).eq(0)) \
                        .updateMask(imageMCD15MOD09.select('LC_Type3').gt(0)) \
                        .rename('LUTout')

In [11]:
# coded LUT for input landsat image
def getLUTL07in(imageMCD15MOD09LC,satFlag):
    
    return imageMCD15MOD09LC.select('SR_B3').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1e3).floor().multiply(1).toInt64() \
                        .add(imageMCD15MOD09LC.select('SR_B4').multiply(2.75e-05).add(0.02).max(0).min(0.999).multiply(1000).floor().multiply(1e6).toInt64()) \
                        .add(imageMCD15MOD09LC.select('VIEW_ZENITH').multiply(0.01).multiply(0.5).multiply(10).floor().multiply(1e9).toInt64()) \
                        .add(ee.Image.constant(0).multiply(0.01).add(180).multiply(0.25).floor().multiply(1e11).toInt64()) \
                        .add(ee.Image.constant(90).subtract(ee.Image(imageMCD15MOD09LC.getNumber('SUN_ELEVATION'))).multiply(0.01).multiply(0.5).multiply(10).floor().multiply(1e14).toInt64()) \
                        .add(ee.Image(imageMCD15MOD09LC.getNumber('SUN_AZIMUTH')).multiply(0.01).add(180).multiply(0.25).floor().multiply  (1e16).toInt64()) \
                        .add(imageMCD15MOD09LC.select('LC_Type3').multiply(1e18).toInt64()) \
                        .updateMask(imageMCD15MOD09LC.select('FparLai_QC').bitwiseAnd(1<<5).eq(satFlag)) \
                        .updateMask(imageMCD15MOD09LC.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                        .updateMask(imageMCD15MOD09LC.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) \
                        .updateMask(imageMCD15MOD09LC.select('FparExtra_QC').bitwiseAnd(1<<1).eq(0)) \
                        .updateMask(imageMCD15MOD09LC.select('LC_Type3').gt(0)) \
                        .rename('LUTL07in')



In [12]:
# matches daily MCD15 values with MOD09 where we are sure there is only one MOD09 in 4day composite
def getDailyMCD15(dayNum,imageMCD15):
    
    # determine mask of valid MCD15data
    maskMCD15 = (imageMCD15.select('FparLai_QC').eq(0)).Or( (imageMCD15.select('FparLai_QC').eq(2)));  
        
    # find coincident MOD09 data and mask 
    MOD09 = ee.ImageCollection('MODIS/006/MOD09GA').filterDate(imageMCD15.get('system:time_start'),imageMCD15.get('system:time_end'))
    
    # create mask for the MOD09 data that includes only one obs per pixel with a valid MOD09 and MCD15 value  then add MCD15 
    maskMOD09 = MOD09.map(lambda image: image.updateMask(maskMCD15).updateMask(image.select('state_1km').eq(72)) \
                                                .updateMask((image.select('QC_500m').eq(1073741824)).Or(image.select('QC_500m').eq(3221225472)))  ) \
                        .select('num_observations_500m') \
                        .sum() \
                        .eq(1)
            
    #mask all of the MOD09 images and add MCD15 bands
    return ee.Image(MOD09.toList(MOD09.size()).get(dayNum)) \
                    .addBands(imageMCD15) \
                    .updateMask(maskMOD09)

In [13]:
def numberOfPixels(img) :
    imgDescription = ee.Algorithms.Describe( img )
    height = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(0)
    width = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(1)
    return  ee.Number( width ).multiply( ee.Number( height ) )

def numberOfPixelsW(img) :
    imgDescription = ee.Algorithms.Describe( img )
    width = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(1)
    return  ee.Number( width )

def numberOfPixelsH(img) :
    imgDescription = ee.Algorithms.Describe( img )
    height = ee.List( ee.Dictionary( ee.List(  ee.Dictionary( imgDescription ).get("bands") ).get(0) ).get("dimensions") ).get(0)
    return  ee.Number(height )


In [14]:
# Aggregates  Landsat image to MODIS MCD15 image  in MCD15 projection and ensures there are sufficient valid landsat and modis pixels 
def matchLCtoMCD15(imageMCD15,imageLC,fracValid, minLandsatPixels, minModisPixels, outputScale):
    
    
    # approximately reduce size of clip regionto ensure we dont encounter out of memory
    # currently we do this by removing outer 25% of landsat image region
    # would be better to remove top and bottom 25% so we sample all view zenith angles
    imageMCD15 = imageMCD15.clip(imageLC.geometry().buffer(-(ee.Number(numberOfPixels(imageMCD15)).getInfo())/4))
    print('numpixels',ee.Number(numberOfPixels(imageMCD15)).getInfo())
    print('numpixelsW',ee.Number(numberOfPixelsW(imageMCD15)).getInfo())
    print('numpixelsH',ee.Number(numberOfPixelsH(imageMCD15)).getInfo())

    
    # Add to the MCD15 image a band that indciates if the MCD15 pixel is unmasked and a band that counts the number of unmasked landsat pixels
    # Then aggregate everything to sum the #valid input pixels at the outputScale (this is done in two steps for landsat )
    countImage = imageMCD15.addBands(ee.Image.constant(1).rename('countMCD')).select('countMCD') \
                           .addBands(imageLC.select('maskLC').rename('countLC') \
                           .clip(imageMCD15.geometry()) \
                           .reproject(crs= imageMCD15.select('Lai').projection(),crsTransform= None,scale=imageLC.projection().nominalScale() ) \
                           .reduceResolution(reducer= ee.Reducer.sum(),maxPixels= 1024,bestEffort=False) \
                           .reproject(crs= imageMCD15.projection().nominalScale())) \
                           .reduceResolution(reducer= ee.Reducer.sum(),maxPixels= 1024,bestEffort=False) \
                           .reproject(crs= imageMCD15.projection().atScale(outputScale))      

    # Add aggregateds LC bands and LC geometry at 500m resolution and then up to the desired outputScale
    imageMCD15 = imageMCD15.addBands(ee.Image(ee.Image(imageMCD15.addBands(imageLC.clip(imageMCD15.geometry()).reproject(crs= imageMCD15.select('Lai').projection(),crsTransform= None,scale=imageLC.projection().nominalScale() ) \
                                                 .reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024,bestEffort=False) \
                                                 .reproject(crs= imageMCD15.projection().nominalScale())) \
                           .reduceResolution(reducer= ee.Reducer.mean(),maxPixels= 1024,bestEffort=False) \
                           .reproject(crs= imageMCD15.projection().atScale(outputScale))))\
                           .set('SUN_AZIMUTH', imageLC.getNumber('SUN_AZIMUTH'))) \
                           .set('SUN_ELEVATION', imageLC.getNumber('SUN_ELEVATION')) 
    

    # Mask the aggregated image keeping only pixels with sufficient valid MCD and LC pixels
    # we do this since we want to avoid values computed by taking the mean over a set of pixels where many are masked
    return imageMCD15.updateMask(countImage.select('countMCD').gte(minModisPixels)).updateMask(countImage.select('countLC').gte(minLandsatPixels)) 



In [15]:
# Match MOD09A1.006 daily reflectance data and MCD12Q1 annual biome type to  MCD15A3H.006 image composite
# masking any dates without exactly one MOD09A1 valid clear sky value and one valid MCD153AH terra retrieval
def matchMOD09A1toMCD15(imageMCD15):
    
    # get date range of MCD15A3Himage
    d1 = ee.Date(imageMCD15.get('system:time_start'));
    d2 = ee.Date(imageMCD15.get('system:time_end'));
    
#    print(d1.getInfo())
#    print(d2.getInfo())
    # find MOD09A1 products within date range of MCD15 and mask them 
    #colMOD09A1 = ee.ImageCollection("MODIS/006/MOD09GA").filterDate(d1,d2).map(lambda image: image.updateMask(image.select('QC_500m').bitwiseAnd(1<<1).eq(0))  \
    #                                                                      .updateMask(image.select('state_1km').bitwiseAnd(1<<0).eq(0)) \
    #                                                                      .updateMask(image.select('state_1km').bitwiseAnd(1<<1).eq(0))) 
#    print(colMOD09A1.size().getInfo())

    # get biome type map
    imageMCD12Q1 = ee.ImageCollection("MODIS/006/MCD12Q1").map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                                                            .filter(ee.Filter.dateRangeContains('daterange', d1)) \
                                                            .first() \
                                                            .select('LC_Type3') 
#    print(imageMCD12Q1.getInfo())

# make mask of pixels containing only one valid cloud free MOD09 500m retrieval
    # and a primary algorithm retyrieval from terra wigthout significant clouds
    # and add MOD09A1 composite created by apply the MCD15 mask to the MOOD9A1 collection and composite it
    # and add biome type
    imageMCD15 = imageMCD15.updateMask( colMOD09A1.count().select('sur_refl_b01').eq(1) ) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<0).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<1).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<2).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<3).eq(0)) \
                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<4).eq(0)) \
                            .updateMask( imageMCD12Q1.select('LC_Type3').gt(0)) \
                            .addBands( imageMCD12Q1 )
    imageMCD15 = imageMCD15.addBands(colMOD09A1.map(lambda image: image.mask(imageMCD15.select('Lai'))).qualityMosaic('sur_refl_b01'))


    
    
    
    
    return imageMCD15

In [16]:
# sample worldwide initialize
maxCloudCover = 10
Lai_ann_L7 = []
LaiStdDev_ann_L7 = []
Fpar_ann_L7 = []
FparStdDev_ann_L7 = []
codeMOD09_ann_L7 = []
codeLC07_ann_L7 = []
Year_ann_L7 = []
Month_ann_L7 = []
Daystart_ann_L7 =  []
Dayend_ann_L7 =  []
Path_ann_L7 = []
Row_ann_L7 = []
Year_ann_L7sat = []
Month_ann_L7sat = []
Daystart_ann_L7sat =  []
Dayend_ann_L7sat =  []
Path_ann_L7sat = []
Row_ann_L7sat = []

deltaRow = 1
deltaPath = 1
deltaDay = 5
unsatLUT = [];
unsatLUTLCinput = [];
unsatLUTinput = [];
unsatLUToutput = [];
satLUT = [];
satLUTLCinput = [];
satLUTinput = [];
satLUToutput = [];
minfracArea = 0.1
minModisPixels = 9
minLandsatPixels = 9*(500/30)*(500/30)*0.9
outputScale = 1500

In [18]:
# sample worldwide 
# should be 233 and 248
# check each year and month
for year in range(2002,2014):
    
    # specify land cover projected into MOD15 projection and aggregated using median to outputScale    
    projectionMCD15=  ee.ImageCollection('MODIS/006/MCD15A3H').first().select('LAI').projection()
    projectionMCD12 =  ee.ImageCollection("MODIS/006/MCD12Q1").first().select('LC_Type3').projection()
    imageMCD12Q1 = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate(ee.Date.fromYMD(year,1,1),ee.Date.fromYMD(year,12,31)).first() \
                     .select('LC_Type3') \
                     .reproject(crs= projectionMCD15,crsTransform= None,scale=projectionMCD12.nominalScale() ) \
                     .reduceResolution(reducer= ee.Reducer.median(),maxPixels= 1024,bestEffort=False) \
                     .reproject(crs= projectionMCD15.atScale(outputScale))        

    # sdample monthly data
    for month in [6,7]:  # [6,7,5,8,4,9,3,10,2,11,1,12]:
        
        #sample WRS path rows
        for path in range(1,233,deltaPath):
            for row in range(1,248,deltaRow):
                
                # sample deltaDay intervals of each month
                for daystart in range (1,25,deltaDay):
                    
                    # find Landsat images for paht , row and date interval between 0 and maxCloudCover
                    imageLC = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterMetadata('WRS_PATH','equals',path) \
                                                                          .filterMetadata('WRS_ROW','equals',row) \
                                                                          .filterDate(ee.Date.fromYMD(year,month,daystart),ee.Date.fromYMD(year,month,daystart).advance(deltaDay,'days')) \
                                                                          .filter(ee.Filter.gte('CLOUD_COVER_LAND',0)).filter(ee.Filter.lte('CLOUD_COVER_LAND',maxCloudCover))
                    
                    # only proceed if there is at least one filtered landsat image (we will only use the first one found)
                    if (imageLC.size().getInfo() > 0): 
                        
                        # only sample the first scene we found in the interval of deltaDay's and add an approximate VIEW_ZENITH band and valid data band
                        imageLC = imageLC.first()
                        imageLC = imageLC.addBands(getVZALC(imageLC).rename('VIEW_ZENITH')) \
                                         .addBands(imageLC.select('QA_PIXEL').bitwiseAnd(1<<6).gt(0).eq(1).rename('maskLC'))
    
    
                        # get date range of imageLC and print it out
                        d1 = ee.Date(imageLC .get('system:time_start'));
                        d2 = ee.Date(imageLC .get('system:time_end'));      
                        print('year',year,'month',month,'path',path,'row',row)
                
                        
                        # get  MOD15 product containing the aquistion of the  Landsat image
                        imageMCD15 = ee.ImageCollection('MODIS/006/MCD15A3H').map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                                                            .filter(ee.Filter.dateRangeContains('daterange', d1)) 
                        
                        #imageMOD09 = ee.ImageCollection("MODIS/006/MOD09GA").map(lambda image: image.set('daterange',ee.DateRange(ee.Date(image.get('system:time_start')),ee.Date(image.get('system:time_end')))))\
                        #                                    .filter(ee.Filter.dateRangeContains('daterange', d1)) 
                        # only proceed if there is a MODIS product for this date                        
                        #if (imageMCD15.size().getInfo() > 0) and (imageMOD09.size().getInfo() > 0) :
                        
                        
                        # only proceed if there is a MCD15 product containing the Landsat image aquisition
                        if (imageMCD15.size().getInfo() > 0)  :

                            # get MCD15 image from collection, add a saturation state band (1=saturated) and clip to landsat geometry and mask to only keep primary algorithm retrievals
                            imageMCD15 = imageMCD15.first().addBands( imageMCD12Q1 ).addBands(imageMCD15.select('FparLai_QC').bitwiseAnd(1<<5).eq(1),'satFlag')
                            imageMCD15 = imageMCD15.clip(imageLC.geometry()) \
                                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<0).eq(0)) \
                                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<2).eq(0)) \
                                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<3).eq(0)) \
                                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<4).eq(0)) \
                                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<6).eq(0)) \
                                            .updateMask( imageMCD15.select('FparLai_QC').bitwiseAnd(1<<7).eq(0)) 
                            
                            # get the fraction of area with unmasked MCD15 data 
                            validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                            totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                            fracArea = validArea / totalArea
                            print('fracAreaMCD15:',fracArea)    
                    
                            # try to include MOD09 data
                            #imageMOD09 = imageMOD09.first().clip(imageLC.geometry())
                            #validArea = ee.Number(ee.Image.pixelArea().mask(imageMOD09.select('QC_500m').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=500).get('area')).getInfo()
                            #totalArea = ee.Number(ee.Image.pixelArea().mask(imageMOD09.select('QC_500m').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=500).get('area')).getInfo()
                            #fracArea = validArea / totalArea
                            #print('fracAreaMOD09:',fracArea) 
                            
                            # mask the MCD15 image to only pixels where there is a valid MOD09 terra retrieval on the overpass day
                            #imageMCD15 = matchMOD09A1toMCD15(imageMCD15.updateMask(imageMOD09.select('QC_500m').bitwiseAnd(1<<0).eq(0))  \
                            #                       .updateMask(imageMOD09.select('state_1km').bitwiseAnd(1<<0).eq(0)) \
                            #                       .updateMask(imageMOD09.select('state_1km').bitwiseAnd(1<<1).eq(0)) )

  
                             # get the fraction of area with unmasked data using one band 
                            #validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=500).get('area')).getInfo()
                            #totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=500).get('area')).getInfo()
                            #fracArea = validArea / totalArea
                            #print('fracAreaCombined:',fracArea)                            


                            # sample if there is sufficient unmasked area
                            if ( fracArea > minfracArea ) :

                                    # aggregate Landsat data to MODIS 500m pixels, smooth non quality bands to designated resolution and mask pixels <fracvalid
                                    imageMCD15 = matchLCtoMCD15(imageMCD15,imageLC,minLandsatPixels,minModisPixels,outputScale)
                                    
                                    # get the fraction of area with unmasked data using one band 
                                    validArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                    totalArea = ee.Number(ee.Image.pixelArea().mask(imageMCD15.select('Lai').unmask().mask()).reduceRegion(reducer=ee.Reducer.sum(),geometry=imageLC.geometry(),scale=projectionMCD15.atScale(outputScale)).get('area')).getInfo()
                                    fracArea = validArea / totalArea
                                    print('fracAreaUnmasked:',fracArea) 
                                    
                                    if ( fracArea > minfracArea ) :
                                        
                                        unsatSample = getLUTout(imageMCD15,0).addBands(getLUTL07in(imageMCD15,0)).sample(scale=projectionMCD15.atScale(outputScale), numPixels=10000, dropNulls=True)
                                        unsatSampleoutput = unsatSample.aggregate_array('LUTout').flatten().getInfo()
                                        unsatSampleLCinput = unsatSample.aggregate_array('LUTL07in').flatten().getInfo()

                                        if len(unsatSampleoutput )>0:
                                            
                                            unsatLUToutput = unsatLUToutput + unsatSampleoutput
                                            unsatLUTLCinput = unsatLUTLCinput + unsatSampleLCinput

                                            nsam = len(unsatSampleoutput)
                                            print('nsamoutput unsat:',nsam,'nsaminput unsat',len(unsatSampleLCinput))
                                            Year_ann_L7.append([year]*nsam)
                                            Month_ann_L7.append([month]*nsam)
                                            Daystart_ann_L7.append([daystart]*nsam)
                                            Path_ann_L7.append([path]*nsam)
                                            Row_ann_L7.append([row]*nsam)
   

                                        satSample = getLUTout(imageMCD15,1).addBands(getLUTL07in(imageMCD15,1)).sample(scale=projectionMCD15.atScale(outputScale), numPixels=10000, dropNulls=True)
                                        satSampleoutput = satSample.aggregate_array('LUTout').flatten().getInfo()
                                        satSampleLCinput = satSample.aggregate_array('LUTL07in').flatten().getInfo()

                                        if len(satSampleoutput )>0:
                                            
                                            satLUToutput = satLUToutput + satSampleoutput
                                            satLUTLCinput = satLUTLCinput + satSampleLCinput

                                            nsam = len(unsatSampleoutput)
                                            print('nsamoutputsat :',nsam,'nsaminput sat ',len(unsatSampleLCinput))
                                            Year_ann_L7sat.append([year]*nsam)
                                            Month_ann_L7sat.append([month]*nsam)
                                            Daystart_ann_L7sat.append([daystart]*nsam)
                                            Path_ann_L7sat.append([path]*nsam)
                                            Row_ann_L7sat.append([row]*nsam)
                                            

                # write list after each month is done
                #with open('E:\\wp3\\modis250\\unsatLUTinputLC07', 'wb') as fp1:
                #    pickle.dump(unsatLUTinput, fp1)
                #with open('E:\\wp3\\modis250\\unsatLUToutputLC07', 'wb') as fp2:
                #    pickle.dump(unsatLUToutput, fp2)
                #with open('E:\\wp3\\modis250\\satLUTinputLC07', 'wb') as fp3:
                #    pickle.dump(satLUTinput, fp3)
                #with open('E:\\wp3\\modis250\\satLUToutputLC077', 'wb') as fp4:
                #    pickle.dump(satLUToutput, fp4)

year 2002 month 6 path 1 row 5
year 2002 month 6 path 1 row 6
year 2002 month 6 path 1 row 69
year 2002 month 6 path 1 row 76
year 2002 month 6 path 1 row 77
year 2002 month 6 path 2 row 4
year 2002 month 6 path 2 row 5
year 2002 month 6 path 2 row 6
year 2002 month 6 path 2 row 7
year 2002 month 6 path 2 row 8
year 2002 month 6 path 2 row 26
year 2002 month 6 path 2 row 27
year 2002 month 6 path 2 row 65
year 2002 month 6 path 2 row 67
year 2002 month 6 path 2 row 68
year 2002 month 6 path 2 row 71
year 2002 month 6 path 2 row 71
year 2002 month 6 path 2 row 72
year 2002 month 6 path 2 row 74
year 2002 month 6 path 2 row 74
year 2002 month 6 path 2 row 75
year 2002 month 6 path 2 row 75
year 2002 month 6 path 2 row 77
year 2002 month 6 path 3 row 16
year 2002 month 6 path 3 row 17
year 2002 month 6 path 4 row 3
year 2002 month 6 path 4 row 4
year 2002 month 6 path 4 row 5
year 2002 month 6 path 4 row 6
year 2002 month 6 path 4 row 27
year 2002 month 6 path 4 row 66
year 2002 month 6 p

KeyboardInterrupt: 